In [ ]:
from PIL import Image
import numpy as np
import diplib
from PIL import ImageFilter
import astroalign as aa
import cv2
import sys
# from common_functions import qgg

%run "common_functions.py"

sys.setrecursionlimit(530000) # override needed for computing midpoints, which uses a recursive function
Image.MAX_IMAGE_PIXELS = 366498276 # override is needed, or else it gives a DecompressionBombError

# cy1_file = 'originals/ImageSLIDE4-CYCLE1.tif'
# cy2_file = 'originals/ImageARRAY4-CYCLE2.tif'
# cy1_shifted_file = "originals/shiftedCycle1.tif"
# cy2_shifted_file = "originals/shiftedCycle2.tif"

cy1_file = 'originals/cycle 1thnksgv.tif'
cy2_file = 'originals/cycle 2thnksgv.tif'
cy1_shifted_file = "originals/shiftedcycle 1thnksgv.tif"
cy2_shifted_file = "originals/shiftedcycle 2thnksgv.tif"

qgg()

In [ ]:
## open tiffs
## lower res

layer_to_align = 0

img_cy1 = Image.open(cy1_file)
img_cy1.seek(layer_to_align) # navigate to brightfield
brightfield_cy1 = np.array(img_cy1)
brightfield_cy1 = (brightfield_cy1/256).astype('uint8') # don't need the whole range of values, so this reduces mem size, + improves access speed

img_cy2 = Image.open(cy2_file)
img_cy2.seek(layer_to_align) # navigate to brightfield
brightfield_cy2 = np.array(img_cy2)
brightfield_cy2 = (brightfield_cy2/256).astype('uint8') 


In [ ]:
# The code seeks to fix the following problem:
# PROBLEM: the two images have different sizes, so when you attempt to scale them down, they get squished
y_cf1, x_cf1 = brightfield_cy1.shape
y_cf2, x_cf2 = brightfield_cy2.shape
max_dim = max(x_cf1, x_cf2, y_cf1, y_cf2)

scale_down_factor = 1 if max_dim < 3000 else round(max_dim / 3000)

# used later!
new_size = max_dim // scale_down_factor
print(new_size)

print(brightfield_cy1.shape, brightfield_cy2.shape)

brightfield_cy1_padded = np.pad(brightfield_cy1, pad_width=((max_dim-y_cf1, 0), (0, max_dim-x_cf1)), mode = 'median')
brightfield_cy2_padded = np.pad(brightfield_cy2, pad_width=((max_dim-y_cf2, 0), (0, max_dim-x_cf2)), mode = 'median')

print(brightfield_cy1_padded.shape)

brightfield_cy1_reduced = cv2.resize(brightfield_cy1_padded, (new_size, new_size)) # 96.5% smaller = faster
brightfield_cy2_reduced = cv2.resize(brightfield_cy2_padded, (new_size, new_size)) # 96.5% smaller = faster

### DONT DELETE!    visual
# c_many([brightfield_cy1, resized_down ], ["Cycle 1 (original)", "Cycle 1 (reduced)"])

c(brightfield_cy1_reduced)

In [ ]:
### clean up
img_cy1 = None
img_cy2 = None
brightfield_cy1 = None
brightfield_cy2 = None
brightfield_cy1_padded = None
brightfield_cy2_padded = None

In [ ]:

bf_cy1_processed = blur_noise_reduction((brightfield_cy1_reduced))
bf_cy1_thresholded = threshold(bf_cy1_processed, 97)


bf_cy2_processed = blur_noise_reduction((brightfield_cy2_reduced))
bf_cy2_thresholded = threshold(bf_cy2_processed, 97)



c(bf_cy2_thresholded)

In [ ]:
%run "common_functions.py"

mp, sizes, im = get_center_points(bf_cy1_thresholded.copy(), min_size=20, )
mp2, sizes2, im2 = get_center_points(bf_cy2_thresholded.copy(), min_size=20)



In [ ]:
cf1_lr = lower_right(mp)
cf1_ul = upper_left(mp)
cf1_center = np.average([cf1_lr, cf1_ul],  axis = 0)
print("original cf1 ", cf1_lr, cf1_ul)


cf2_lr = lower_right(mp2)
cf2_ul = upper_left(mp2)
cf2_center = np.average([cf2_lr, cf2_ul],  axis = 0)
print("original cf2 ", cf2_lr, cf2_ul)

translation_cf2_to_cf1 = [cf1_center[0] - cf2_center[0],  cf1_center[1] - cf2_center[1]]

print("\nTRANSLATION NEEDED", translation_cf2_to_cf1, "\n")

translated_cf2_lr = [cf2_lr[0] + translation_cf2_to_cf1[0], cf2_lr[1] + translation_cf2_to_cf1[1]]
translated_cf2_ul = [cf2_ul[0] + translation_cf2_to_cf1[0], cf2_ul[1] + translation_cf2_to_cf1[1]]
t_cf2_center = np.average([translated_cf2_lr, translated_cf2_ul],  axis = 0)

print("translated c2", translated_cf2_lr, translated_cf2_ul)
print("and new center is", t_cf2_center)


### used later for the cropping:
### the math here is basically like on the unit circle, 
### where the hyp = sqrt(2) at 45 degrees, and b = c  so a^2 = 2b^2
### so b = sqrt( a^2/2 )

cf1_width = math.sqrt ( (math.dist(cf1_ul, cf1_center)**2 / 2) )
cf2_width = math.sqrt ( (math.dist(cf2_ul, cf1_center)**2 / 2) )


print(cf1_width)
print(cf2_width)

### take the bigger of the two, make it an int, and add nessecary padding
padding = 50
FINAL_CROP_WIDTH = (int(max(cf1_width, cf2_width)) + padding) * 10


# print(t_cf2_center)
# image1 = cv2.line(im.copy(), cf1_ul, np.array(cf1_center).astype("int"), 120, 4) 
# c(image1)



In [ ]:
# print(t_cf2_center)
# image2 = cv2.line(im2.copy(), cf2_lr, cf2_ul, 120, 4) 
# c(image2)

In [ ]:
import math

def find_slope(x1, y1, x2, y2): # Line slope given two points:
    return (y2-y1)/(x2-x1)

def find_angle(s1, s2): 
    return math.degrees(math.atan((s2-s1)/(1+(s2*s1))))

lineA = (t_cf2_center, translated_cf2_ul)
lineB = (cf1_center, cf1_ul)

print(lineA)
print(lineB)

slope1 = find_slope(lineA[0][0], lineA[0][1], lineA[1][0], lineA[1][1])
slope2 = find_slope(lineB[0][0], lineB[0][1], lineB[1][0], lineB[1][1])

FINAL_ANGLE = find_angle(slope1, slope2)
print('Angle in degrees = ', FINAL_ANGLE)

def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    
    # degrees = angle

    angle = angle*(math.pi/180)

    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy



In [ ]:
c(im) # original 

In [ ]:
c(im2)

In [ ]:
##
# 
#     THIS WARPS THE DOWNSCALED IMAGE
# 
# ## FIRST, SHIFT
cp = im2.copy() 
print(translation_cf2_to_cf1)
translation_matrix = np.float32([ [1,0,translation_cf2_to_cf1[0]], [0,1,translation_cf2_to_cf1[1]] ])
img_translation = cv2.warpAffine(cp.copy(), translation_matrix, (new_size,new_size))

print("shift success!")
c(img_translation)



# +----------------------------------+
# |                                  |
# |   New Step added Nov 29th:       |
# |   Make two squares orthagonal.   |
# |                                  |
# |   This helps with the            |
# |   micro-alignment step later.    |
# |                                  |
# +----------------------------------+


lineAOrth = (lineB)
lineBOrth = [lineB[0], lineB[0] - 400]

slope1Orth = find_slope(lineAOrth[0][0], lineAOrth[0][1], lineAOrth[1][0], lineAOrth[1][1])
slope2Orth = find_slope(lineBOrth[0][0], lineBOrth[0][1], lineBOrth[1][0], lineBOrth[1][1])

angleOrth = -find_angle(slope1Orth, slope2Orth)

warp_dst = im.copy()

center = t_cf2_center

print('Orth Angle in degrees = ', angleOrth)


# +----------------------------------+


## THEN, ROTATE
warp_dst = img_translation

center = t_cf2_center
angle = -(FINAL_ANGLE + .1) + angleOrth
print("angle!", angle)
scale = 1
rot_mat = cv2.getRotationMatrix2D( center, angle, scale )
print(t_cf2_center)
warp_rotate_dst = cv2.warpAffine(warp_dst, rot_mat, (warp_dst.shape[1], warp_dst.shape[0]))

c(warp_rotate_dst)

In [ ]:

# # +----------------------------------+
# # |                                  |
# # |   New Step added Nov 29th:       |
# # |   Make two squares orthagonal.   |
# # |                                  |
# # |   This helps with the            |
# # |   micro-alignment step later.    |
# # |                                  |
# # +----------------------------------+


# def line(p1, p2):
#     A = (p1[1] - p2[1])
#     B = (p2[0] - p1[0])
#     C = (p1[0]*p2[1] - p2[0]*p1[1])
#     return A, B, -C

# def intersection(L1, L2):
#     D  = L1[0] * L2[1] - L1[1] * L2[0]
#     Dx = L1[2] * L2[1] - L1[1] * L2[2]
#     Dy = L1[0] * L2[2] - L1[2] * L2[0]
#     if D != 0:
#         x = Dx / D
#         y = Dy / D
#         return x,y
#     else:
#         return False
    


# lineAOrth = (lineB)
# lineBOrth = [lineB[0], lineB[0] - 400]

# print(lineAOrth)
# print(lineBOrth)

# # intersection()

# # slope1Orth = find_slope(lineAOrth[1][0], lineAOrth[1][1], lineAOrth[0][0], lineAOrth[0][1])
# # slope2Orth = find_slope(lineBOrth[1][0], lineBOrth[1][1], lineBOrth[0][0], lineBOrth[0][1])

# slope1Orth = find_slope(lineAOrth[0][0], lineAOrth[0][1], lineAOrth[1][0], lineAOrth[1][1])
# slope2Orth = find_slope(lineBOrth[0][0], lineBOrth[0][1], lineBOrth[1][0], lineBOrth[1][1])

# angleOrth = -  find_angle(slope1Orth, slope2Orth)

# warp_dst = im.copy()


# # c(cv2.line((warp_dst),lineAOrth[0],lineAOrth[1],(170,0,0),5) )
# # c(cv2.line((warp_dst),lineBOrth[0],lineBOrth[1],(170,0,0),5) )
# center = t_cf2_center

# print('Angle in degrees = ', angleOrth)

# scale = 1
# rot_mat = cv2.getRotationMatrix2D( center, angleOrth, scale )

# warp_rotate_dst2 = cv2.warpAffine(warp_dst, rot_mat, (warp_dst.shape[1], warp_dst.shape[0]))


# FINAL_ANGLE = angle - angleOrth
# print(FINAL_ANGLE)
# c(warp_rotate_dst2)

In [ ]:
# 1/0

In [ ]:
###
#
#     THIS WARPS THE RRREEEEEEEAAAALLLL IMAGE 
# 
###
# open files
bf_layer = 2 # PRESET NEEDE

img_cy1 = Image.open(cy1_file)
img_cy1.seek(bf_layer) # navigate to brightfield
cy1_array = np.array(img_cy1)
print("cy1_array before padding", cy1_array.shape)

img_cy2 = Image.open(cy2_file)
img_cy2.seek(bf_layer) # navigate to brightfield_cy1
cy2_array = np.array(img_cy2)
print("cy2_array before padding", cy2_array.shape)
# end open files

### PADDING STEP
# pad image to square 
y_cf1, x_cf1 = cy1_array.shape
y_cf2, x_cf2 = cy2_array.shape
max_dim = max(x_cf1, x_cf2, y_cf1, y_cf2)

cy1_array = np.pad(cy1_array, pad_width=((max_dim-y_cf1, 0), (0, max_dim-x_cf1)), mode = 'median')
print("cy1_array after padding ", cy1_array.shape)

cy2_array = np.pad(cy2_array, pad_width=((max_dim-y_cf2, 0), (0, max_dim-x_cf2)), mode = 'median')
print("cy2_array after padding ", cy2_array.shape)
### END PADDING STEP




In [ ]:
### SCALE TRANSLATION STEP
print(scale_down_factor)
print()
if scale_down_factor == 1:  
    print("triggered")
    scale_down_factor = (max_dim/new_size)
    print(scale_down_factor)
print(scale_down_factor)
upscaled_translation = np.array(translation_cf2_to_cf1) * scale_down_factor
upscaled_center = t_cf2_center * scale_down_factor


print("translate from", translation_cf2_to_cf1, "to", upscaled_translation)
print("rotational center from", t_cf2_center, "to", upscaled_center)
print(FINAL_ANGLE)
### END SCALE TRANSLATION STEP


### Now shift cy2
# translating
translation_matrix = np.float32([ [1,0,upscaled_translation[0]], [0,1,upscaled_translation[1]] ])
img_translation = cv2.warpAffine(cy2_array, translation_matrix, cy2_array.shape)

# rotating
rot_mat = cv2.getRotationMatrix2D( upscaled_center , -(FINAL_ANGLE + .1) + angleOrth, 1 )
cy2_pre_crop = cv2.warpAffine(img_translation, rot_mat, (img_translation.shape[1], img_translation.shape[0]))
cy2_cropped = cy2_pre_crop[int(upscaled_center[1] - FINAL_CROP_WIDTH):int(upscaled_center[1] +FINAL_CROP_WIDTH), 
                                   int( upscaled_center[0] - FINAL_CROP_WIDTH):int(upscaled_center[0] +FINAL_CROP_WIDTH)]
print(cy2_cropped.shape)
c(cy2_cropped).save(cy2_shifted_file)
### Added Nov 29th, Nov 30th: 
### crop step, to reduce black stuff...


## Now Shift Cy1

rot_mat = cv2.getRotationMatrix2D( upscaled_center , angleOrth, 1 )
cy1_pre_crop = cv2.warpAffine(cy1_array, rot_mat, (img_translation.shape[1], img_translation.shape[0]))
cy1_cropped = cy1_pre_crop[int(upscaled_center[1] - FINAL_CROP_WIDTH):int(upscaled_center[1] +FINAL_CROP_WIDTH), 
                                   int( upscaled_center[0] - FINAL_CROP_WIDTH):int(upscaled_center[0] +FINAL_CROP_WIDTH)]
print(cy1_cropped.shape)
c(cy1_cropped).save(cy1_shifted_file)

In [ ]:
print(upscaled_center[1] - FINAL_CROP_WIDTH,upscaled_center[1] +FINAL_CROP_WIDTH, upscaled_center[0] - FINAL_CROP_WIDTH, upscaled_center[0] +FINAL_CROP_WIDTH)

In [ ]:
###
#
#        IGNORE BUT DON't DELETE... USED FOR EASY VIEWING
# 
# ##


img_cy1 = Image.open(cy1_shifted_file)
brightfield_cy1 = np.array(img_cy1)
brightfield_cy1 = (brightfield_cy1/256).astype('uint8') # don't need the whole range of values, so this reduces mem size, + improves access speed
brightfield_cy1_reduced = cv2.resize(brightfield_cy1, (np.array(brightfield_cy1.T.shape) // 10) ) # 96.5% smaller = faster

c(brightfield_cy1_reduced)